<a href="https://colab.research.google.com/github/Billy-24/my_mathematics_repo_temp/blob/main/PROJET_final_loan_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import io
uploaded = files.upload()
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
data = pd.read_csv(io.BytesIO(uploaded[
'train_u6lujuX_CVtuZ9i.csv']), encoding='ISO-8859-1')
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe(include="all")

In [ ]:
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data)
cat_data

In [ ]:

#Renseigner les valeurs manquantes
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data).transpose()
num_data=pd.DataFrame(num_data).transpose()


In [ ]:

# Un test pour voir si on retrouve encore des NAN dans le dataset
cat_data['Gender'].value_counts()

In [ ]:
# pour les valeurs manquantes dans les variables cathégoriques, on doit les remplacer par le Mode
cat_data=cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
cat_data.isnull().sum().any()

In [ ]:
cat_data

In [ ]:

num_data

In [ ]:
cat_data['Dependents'].value_counts()

In [ ]:
#remplaçons les valeur manquante numérique par le mean
num_data.fillna(method = 'bfill' , inplace= True)
num_data.isnull().sum().any()

In [ ]:
num_data

In [ ]:
cat_data.head()

In [ ]:
#fusionner dataset à cette étape (combined_data)
combined_data = pd.concat([cat_data, num_data], axis=1)

combined_data

In [ ]:
"""
fusionner dataset à cette étape (combined_data)

ensuite: isoler Features dans X (combined_data.colonnes_concernée), et target dans y (combined_data.colonne_concernée)

train test split

preprocessing(encoder les X et y)

Machine Learning
"""

In [ ]:
# Supposons que combined_data est votre DataFrame contenant les données combinées
target_values = {'Y': 1, 'N': 0}

# Créez un nouveau DataFrame avec la colonne 'Loan_Status' modifiée
modified_data = combined_data.copy()
modified_data['Loan_Status'] = modified_data['Loan_Status'].replace(target_values)

In [ ]:
modified_data

In [ ]:
#ensuite: isoler Features dans X (combined_data.colonnes_concernée), et target dans y (combined_data.colonne_concernée)
# Separate target variable Y from features X
## Choose the columns you want to have as your features
features_list = ['Gender', 'Married', 'Dependents','Education','Self_Employed','ApplicantIncome',	'CoapplicantIncome',	'LoanAmount',	'Loan_Amount_Term',	'Credit_History',	'Property_Area']
print("Splitting dataset into X and y...")
X = modified_data.loc[:,features_list] # We add feature_list into our loc

y = modified_data.loc[:,"Loan_Status"] # We set "Loan_Status" as the target variable
print("...Done.")

In [ ]:
from sqlalchemy.sql.expression import true
#Supprimons Loan_ID
modified_data.drop('Loan_ID', axis=1, inplace=True)

In [ ]:
# Créez un nuage de points avec 'ApplicantIncome' sur l'axe des x
plt.scatter(range(len(modified_data['ApplicantIncome'])), modified_data['ApplicantIncome'])
plt.show()

In [ ]:
modified_data

In [ ]:
# Définissez une limite supérieure pour les valeurs acceptables
limite_supérieure = 30000
modified_data= modified_data[modified_data['ApplicantIncome'] <= limite_supérieure]

In [ ]:
# Scatter plot avant la suppression
plt.scatter(range(len(modified_data['ApplicantIncome'])), modified_data['ApplicantIncome'])

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(modified_data.isna(), cbar=False)

In [ ]:
### Training pipeline ###
print("--- Training pipeline ---")
print()

# Encoding categorical features and standardizing numeric features

print("#### X_train BEFORE preprocessing ####")
print(modified_data.head())
print()

print("Encoding categorical features and standardizing numerical features...")
## First let's import libraries
## StandardScaler to scale data (i.e apply Z-score)
## OneHotEncoder to encode categorical variables


numeric_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History'] # Choose which column index we are going to scale
numeric_transformer = StandardScaler()

categorical_features = ['Gender', 'Married', 'Dependents', 'Education','Self_Employed','Property_Area'] # Choose which column index we are going to encode
categorical_transformer = OneHotEncoder(drop="first")

# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features), ('num', numeric_transformer, numeric_features)])
# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
        ]
    )



In [ ]:
print("Splitting modified_data into train set and test set...")
## Then we use train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,stratify=y)

In [ ]:
X_train = feature_encoder.fit_transform(X_train)
print("...Done.")
print("#### X_train AFTER preprocessing ####")
print(X_train[0:5,:]) # print first 5 rows (not using iloc since now X_train became a numpy array)
print()


In [ ]:
#  CREONS NOTRE MODELE
## Training model
print("Training model...")
classifier = LogisticRegression(random_state = 0) # Instanciate model
classifier.fit(X_train, y_train) # Fit model
print("...Done.")

In [ ]:
# Predictions on training set
print("Predictions on train set...")
y_train_pred = classifier.predict(X_train)
print("...Done.")
print()

print("#### First five predictions on TRAIN set ####")
print(y_train_pred[0:5])

In [ ]:
### Test pipeline ###
print("--- Test pipeline ---")

# Encoding categorical features and standardizing numeric features
print("Encoding categorical features and standardizing numerical features...")
print()

X_test = feature_encoder.transform(X_test)
print("...Done.")

print("#### X_test AFTER preprocessing ####")
print(X_test[0:5,:])
print()

In [ ]:
# Predictions on training set
print("Predictions on test set...")
y_test_pred = classifier.predict(X_test)
print("...Done.")
print()

print("#### First five predictions on TEST set ####")
print(y_train_pred[0:5])

In [ ]:
### VISUALISATION DE NOS RESULTAT ET EVALUATION DE NOTRE MODELE POUR VOIR les scores de précision sur l'ensemble du test

### Assessment of performances ###
print("--- Assessment of performances ---")

# Plot confusion matrix
cm = confusion_matrix(y_train, y_train_pred, labels=classifier.classes_)
cm_display = ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)
cm_display.ax_.set_title("Confusion matrix on train set") # Simply to set a title
plt.show() # Show graph
print("accuracy-score on train set : ", classifier.score(X_train, y_train))


cm = confusion_matrix(y_test, y_test_pred, labels=classifier.classes_)
cm_display = ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
cm_display.ax_.set_title("Confusion matrix on train set ") # Simply to set a title
plt.show() # Show graph
print("accuracy-score on test set : ", classifier.score(X_test, y_test))


In [ ]:
modified_data['Loan_Status'].value_counts()

In [ ]:

## VISUALISONS L'IMPACT DE CHAQUE VARIBLE SUR L'ACCORD OU NON DU CREDIT
# En fonction du crédit historique
grid = sns.FacetGrid(modified_data, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot, 'Credit_History')
grid.set_axis_labels('Credit History', 'Count')
grid.set_titles('{col_name} Status')
plt.show()

In [ ]:
##En fonction du SEXE
grid = sns.FacetGrid(modified_data, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot,'Gender')
grid.set_axis_labels('Gender', 'Count')
grid.set_titles('{col_name} Status')

In [ ]:
##En fonction des Diplomes
##En fonction du SEXE
grid = sns.FacetGrid(modified_data, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot,'Education')
grid.set_axis_labels('Education', 'Count')
grid.set_titles('{col_name} Status')

In [ ]:
## en fonction des revenus du demandeur
plt.scatter(modified_data['ApplicantIncome'],modified_data ['Loan_Status'])

In [ ]:
data.dtypes.value_counts().plot.pie()
#Calculer les pourcentages
data_types_counts = data.dtypes.value_counts()

#Afficher les pourcentages sous forme de graphique en secteurs (pie chart)
data_types_counts.plot.pie(autopct='%1.1f%%')
plt.title("Pourcentage de chaque type de données")
plt.show()